In [55]:
import torch
import torch_geometric
import networkx as nx

from torch_geometric.nn import MLP
from torch_geometric.datasets import Planetoid

In [56]:
dataset = Planetoid(root='data/Cora', name='Cora')
G = torch_geometric.utils.to_networkx(dataset[0], to_undirected=True)

In [57]:
degree_dict = nx.degree_centrality(G)
betweenness_dict = nx.betweenness_centrality(G)
eigenvector_dict = nx.eigenvector_centrality(G, max_iter=1000)
closeness_dict = nx.closeness_centrality(G)
clustering_dict = nx.clustering(G)

In [58]:
data = dataset[0]

In [59]:
indices = range(data.num_nodes)
degree_feat = torch.tensor([degree_dict[i] for i in indices]).unsqueeze(1)
betweenness_feat = torch.tensor([betweenness_dict[i] for i in indices]).unsqueeze(1)
eigenvector_feat = torch.tensor([eigenvector_dict[i] for i in indices]).unsqueeze(1)
closeness_feat = torch.tensor([closeness_dict[i] for i in indices]).unsqueeze(1)
clustering_feat = torch.tensor([clustering_dict[i] for i in indices]).unsqueeze(1)

data.x = torch.cat([data.x,degree_feat,betweenness_feat,eigenvector_feat,closeness_feat,clustering_feat], dim=1)

In [60]:
data

Data(x=[2708, 1438], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [61]:
train_mask = data.train_mask
test_mask = data.test_mask
val_mask = data.val_mask

In [142]:
model = MLP(in_channels=data.x.size(1), hidden_channels=32, out_channels=dataset.num_classes, num_layers=2, dropout=0.4)

In [143]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

In [144]:
verbose=1

In [145]:
for i in range(1, 201):
    model.train()
    optimizer.zero_grad()
    pred=model(data.x[train_mask])
    loss=criterion(pred, data.y[train_mask])
    loss.backward()
    optimizer.step()
    if i %10 ==0 and verbose > 0:
        print(f"Epoch: {i}; Training loss: {loss}")

    if i %10 ==0 and verbose > 0:
        model.eval()
        with torch.no_grad():
            pred=model(data.x[val_mask])
            loss=criterion(pred, data.y[val_mask]) 
            print(f"Epoch: {i}; Val loss: {loss}")      

Epoch: 10; Training loss: 0.5711208581924438
Epoch: 10; Val loss: 1.6899352073669434
Epoch: 20; Training loss: 0.1952216923236847
Epoch: 20; Val loss: 1.4549846649169922
Epoch: 30; Training loss: 0.13749440014362335
Epoch: 30; Val loss: 1.4336274862289429
Epoch: 40; Training loss: 0.08225499093532562
Epoch: 40; Val loss: 1.4863533973693848
Epoch: 50; Training loss: 0.05347158759832382
Epoch: 50; Val loss: 1.5378046035766602
Epoch: 60; Training loss: 0.03344025835394859
Epoch: 60; Val loss: 1.5725793838500977
Epoch: 70; Training loss: 0.028133947402238846
Epoch: 70; Val loss: 1.5890628099441528
Epoch: 80; Training loss: 0.05381563678383827
Epoch: 80; Val loss: 1.6063263416290283
Epoch: 90; Training loss: 0.010545135475695133
Epoch: 90; Val loss: 1.6337193250656128
Epoch: 100; Training loss: 0.026129113510251045
Epoch: 100; Val loss: 1.6395823955535889
Epoch: 110; Training loss: 0.020222874358296394
Epoch: 110; Val loss: 1.6626200675964355
Epoch: 120; Training loss: 0.02631177380681038
E

In [146]:
model.eval()
with torch.no_grad():
    logits = model(data.x)
    test_preds = logits[test_mask].argmax(dim=1)
    test_acc = (test_preds == data.y[test_mask]).float().mean().item()
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.5350
